## 1. Imports

In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df_one = pd.read_csv('./Job_data/DataAnalyst.csv')
print(df_one.shape)
df_one.head(3)

(2253, 16)


,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True
1,1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,-1
2,2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy,-1


In [3]:
df_two = pd.read_csv('./Job_data/DataScientist.csv')
print(df_two.shape)
df_two.head(3)

(3909, 17)


,Unnamed: 0,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1
1,1,1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1
2,2,2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,True


There were three other datasets that I found, but they were dirtier and I figured these two sets were plenty enough without needing to burn a bunch of time on cleaning.

## Labeling, Dropping & Concatting

In [4]:
df_one['Org_DF'] = 'Data Analyst'
df_two['Org_DF'] = 'Data Scientist'

df_one.drop('Unnamed: 0', axis = 1, inplace = True)

df_two.drop(['Unnamed: 0','index'], axis = 1, inplace = True)

list_dfs = [df_one, df_two]
df = pd.concat(list_dfs)

df.reset_index(inplace = True)

## Filling Na's (listed in data as -1)

In [5]:
df['Easy Apply'].value_counts()

-1      5918
True     244
Name: Easy Apply, dtype: int64

In [6]:
df['Easy Apply'].replace('-1', False, inplace = True)

In [7]:
# This will both replace the Na's as well as 
def funt_min(x, regex = r'\$\d+K', front = '$'):
    cell = re.search(regex, x)
    if type(cell)== type(None):
        cell = -1
    else:
        cell = cell.group()
        cell = cell.strip('K').strip(front)
        cell = int(cell)# * 1000
    return cell

df['Salary_min_grand'] = df['Salary Estimate'].map(lambda x: funt_min(x))

df['Salary_max_grand'] = df['Salary Estimate'].map(lambda x: funt_min(x, 
                                                   regex = r'-\$\d+K', front = '-$'))

In [8]:
df['Revenue'].replace('-1', 'Unknown / Non-Applicable', inplace = True)

In [9]:
df.drop(1860, inplace = True)#.loc[1855:1865]

In [10]:
def fun_map(x):
    if x[-4:-3] == "\n":
        x = x[:-4]
    return x
df['Company Name'] = df['Company Name'].map(lambda x: fun_map(x))

## Bonus Feature Engineering

In [11]:
df['Salary_avg_gr'] = (df['Salary_min_grand'] + df['Salary_max_grand'])/2

### Dropping things

In [14]:
df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,Org_DF,Salary_min_grand,Salary_max_grand,Salary_avg_gr
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True,Data Analyst,37,66,51.5
1,1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,False,Data Analyst,37,66,51.5
2,2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy,False,Data Analyst,37,66,51.5
3,3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),-1,False,Data Analyst,37,66,51.5
4,4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),DraftKings,True,Data Analyst,37,66,51.5


In [20]:
!pip install time

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [21]:
# ALWAYS UPDATE THE DATE
#df.to_csv('./Job_data/jobs_#10_13.csv', index = False)

#df.to_excel('./Job_data/jobs_#10_13.xlsx', index = False)